In [1]:
import pandas as pd
import numpy as np
from translate import translator
import time
import random
from babel import Locale
import json
import operator
from prompter import yesno

In [2]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet

[nltk_data] Error loading wordnet: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>


In [3]:
path = "Data/"
path_foreigners = "Resident foreigners on 1st January - Citizenship/resident_foreigners.csv"
path_resident = "Resident population  on 1st January /resident.csv"

In [4]:
resident_foreigners = pd.read_table(path+path_foreigners)
resident_foreigners = resident_foreigners.drop_duplicates()
resident_foreigners = resident_foreigners.drop("Tipo di indicatore demografico", axis = 1)
resident_foreigners.columns = ["Territorio", "Cittadinanza", "Periodo", "Sesso", "Numero"]
resident_foreigners.head()

,Territorio,Cittadinanza,Periodo,Sesso,Numero
0,Italia,Andorra,2013,maschi,0
1,Italia,Andorra,2014,maschi,2
2,Italia,Andorra,2015,maschi,1
3,Italia,Andorra,2016,maschi,1
4,Italia,Andorra,2017,maschi,1


In [7]:
# Is not useful to keep the info about the ages, since it's an info not available for the foreigners table
resident = pd.read_table(path+path_resident)
resident = resident.drop_duplicates()
resident = resident[resident["Età"] == "totale"]
resident = resident.drop(["Tipo di indicatore demografico", "Stato civile", "Età"], axis = 1)
resident.columns = ["Territorio", "Periodo", "Sesso", "Numero"]
resident.head()

,Territorio,Periodo,Sesso,Numero
179734,Piemonte,2013,maschi,2110576
179735,Piemonte,2014,maschi,2146570
179736,Piemonte,2015,maschi,2140294
179737,Piemonte,2016,maschi,2131892
179738,Piemonte,2017,maschi,2129403


In [8]:
provincia_regione = pd.read_table(path+"regioni.csv", sep = "\t")
provincia_regione.head()

,Provincia,Regione
0,Torino,Piemonte
1,Vercelli,Piemonte
2,Novara,Piemonte
3,Cuneo,Piemonte
4,Asti,Piemonte


In [9]:
regione_zona = pd.read_table(path+"territori.csv", sep = "\t")
regione_zona.head()

,Regione,Zona
0,Abruzzo,Sud
1,Basilicata,Sud
2,Calabria,Sud
3,Campania,Sud
4,Emilia-Romagna,Nord-est


Check if the aggragation is right.. that is: 
- the sum over the municipalities (comuni) conrespond to the aggragation value at the region (regione) level;
- the sum over the regions conrespond to the aggregation value at the zone (nord, sud, etc..) level;
- for each tuple (Territorio, Cittadinanza, Anno) if maschio + femmina = totale.

Check both the resident and the resident_foreigners

In [10]:
regioni = list(set(regione_zona["Regione"]))
zone = list(set(regione_zona["Zona"]))
territori = list(set(resident_foreigners["Territorio"]))
years = list(set(resident_foreigners["Periodo"]))
origine = list(set(resident_foreigners["Cittadinanza"]))

### Resident Foreigners Check

In [11]:
temp = resident_foreigners.copy()
# keep only the municipalities and the region data
territori_1 = list(set(list(provincia_regione["Provincia"]) + list(provincia_regione["Regione"])))
temp = temp.loc[[i in territori_1 for i in temp["Territorio"]]]
# do not consider the Valle d'Aosta / Vallée d'Aoste data since the aggregate and the singular data coincide
temp = temp[temp["Territorio"] != "Valle d'Aosta / Vallée d'Aoste"]
# change the sign to the "total" elements
temp["Numero"][[i in list(set(provincia_regione["Regione"])) for i in temp["Territorio"]]] = -temp["Numero"][[i in list(set(provincia_regione["Regione"])) for i in temp["Territorio"]]]

regioni = list(set(provincia_regione["Regione"]))
province = list(set(provincia_regione["Provincia"]))
# add the regione field, so we can aggregate on it
temp["Regione"] = [provincia_regione["Regione"][provincia_regione["Provincia"] == i].values[0] if i in province else i for i in temp["Territorio"]]

# sum all over the same (Regione, Cittadinanza, Periodo, Sesso)
temp['totale'] = temp.groupby(["Regione", "Cittadinanza", "Periodo", "Sesso"])['Numero'].transform(np.sum)

# the sum should always be 0
if len(list(set(temp["totale"]))) == 1 and list(set(temp["totale"]))[0] == 0:
    print("Region-Aggregation works on 'resident_foreigners'!")

Region-Aggregation works on 'resident_foreigners'!


In [12]:
temp = resident_foreigners.copy()
# keep only the municipalities and the region data
territori_1 = list(set(list(regione_zona["Regione"]) + list(regione_zona["Zona"])))
temp = temp.loc[[i in territori_1 for i in temp["Territorio"]]]

# change the sign to the "total" elements
temp["Numero"][[i in list(set(regione_zona["Zona"])) for i in temp["Territorio"]]] = -temp["Numero"][[i in list(set(regione_zona["Zona"])) for i in temp["Territorio"]]]

regioni = list(set(provincia_regione["Regione"]))
zone = list(set(regione_zona["Zona"]))

# add the regione field, so we can aggregate on it
temp["Zona"] = [regione_zona["Zona"][regione_zona["Regione"] == i].values[0] if i in regioni else i for i in temp["Territorio"]]

# sum all over the same (Regione, Cittadinanza, Periodo, Sesso)
temp['totale'] = temp.groupby(["Zona", "Cittadinanza", "Periodo", "Sesso"])['Numero'].transform(np.sum)

# the sum should always be 0
if len(list(set(temp["totale"]))) == 1 and list(set(temp["totale"]))[0] == 0:
    print("Zone-Aggregation works on 'resident_foreigners'!")

Zone-Aggregation works on 'resident_foreigners'!


In [13]:
temp = resident_foreigners.copy()
# change the sign to the "total" elements
temp["Numero"][temp[temp["Sesso"] == "totale"].index] = -temp["Numero"][temp[temp["Sesso"] == "totale"].index]
# sum all over the same (Territorio, Cittadinanza, Periodo)
temp['totale'] = temp.groupby(["Territorio", "Cittadinanza", "Periodo"])['Numero'].transform(np.sum)

# the sum should always be 0
if len(list(set(temp["totale"]))) == 1 and list(set(temp["totale"]))[0] == 0:
    print("Aggregation maschio + femmina works on 'resident_foreigners'!")

Aggregation maschio + femmina works on 'resident_foreigners'!


/home/sara/myenv/lib64/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### Resident Check

In [14]:
temp = resident.copy()
# keep only the municipalities and the region data
territori_1 = list(set(list(provincia_regione["Provincia"]) + list(provincia_regione["Regione"])))
temp = temp.loc[[i in territori_1 for i in temp["Territorio"]]]
# do not consider the Valle d'Aosta / Vallée d'Aoste data since the aggregate and the singular data coincide
temp = temp[temp["Territorio"] != "Valle d'Aosta / Vallée d'Aoste"]
# change the sign to the "total" elements
temp["Numero"][[i in list(set(provincia_regione["Regione"])) for i in temp["Territorio"]]] = -temp["Numero"][[i in list(set(provincia_regione["Regione"])) for i in temp["Territorio"]]]

regioni = list(set(provincia_regione["Regione"]))
province = list(set(provincia_regione["Provincia"]))
# add the regione field, so we can aggregate on it
temp["Regione"] = [provincia_regione["Regione"][provincia_regione["Provincia"] == i].values[0] if i in province else i for i in temp["Territorio"]]

# sum all over the same (Regione, Periodo, Sesso)
temp['totale'] = temp.groupby(["Regione", "Periodo", "Sesso"])['Numero'].transform(np.sum)

# the sum should always be 0
if len(list(set(temp["totale"]))) == 1 and list(set(temp["totale"]))[0] == 0:
    print("Region-Aggregation works on 'resident'!")

Region-Aggregation works on 'resident'!


In [15]:
temp = resident.copy()
# keep only the region and the zone data
territori_1 = list(set(list(regione_zona["Regione"]) + list(regione_zona["Zona"])))
temp = temp.loc[[i in territori_1 for i in temp["Territorio"]]]

# change the sign to the "total" elements
temp["Numero"][[i in list(set(regione_zona["Zona"])) for i in temp["Territorio"]]] = -temp["Numero"][[i in list(set(regione_zona["Zona"])) for i in temp["Territorio"]]]

regioni = list(set(provincia_regione["Regione"]))
zone = list(set(regione_zona["Zona"]))

# add the regione field, so we can aggregate on it
temp["Zona"] = [regione_zona["Zona"][regione_zona["Regione"] == i].values[0] if i in regioni else i for i in temp["Territorio"]]

# sum all over the same (Regione, Periodo, Sesso)
temp['totale'] = temp.groupby(["Zona", "Periodo", "Sesso"])['Numero'].transform(np.sum)

# the sum should always be 0
if len(list(set(temp["totale"]))) == 1 and list(set(temp["totale"]))[0] == 0:
    print("Zone-Aggregation works on 'resident'!")

Zone-Aggregation works on 'resident'!


In [16]:
temp = resident.copy()
# change the sign to the "total" elements
temp["Numero"][temp[temp["Sesso"] == "totale"].index] = -temp["Numero"][temp[temp["Sesso"] == "totale"].index]
# sum all over the same (Territorio, Periodo)
temp['totale'] = temp.groupby(["Territorio", "Periodo"])['Numero'].transform(np.sum)

# the sum should always be 0
if len(list(set(temp["totale"]))) == 1 and list(set(temp["totale"]))[0] == 0:
    print("Aggregation maschio + femmina works on 'resident'!")

Aggregation maschio + femmina works on 'resident'!


/home/sara/myenv/lib64/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


The aggragation checks are verified. Thus, it's possible to drop duplication to get the normal form.

### .. to Normal Form

In [45]:
resident_foreigners_norm = resident_foreigners.copy() 

# drop region and zone data
territori_1 = list(regione_zona["Regione"].values) + list(regione_zona["Zona"].values) + ["Italia"]
resident_foreigners_norm = resident_foreigners_norm.loc[[i not in territori_1 for i in resident_foreigners_norm["Territorio"]]]
resident_foreigners_norm = resident_foreigners_norm[resident_foreigners_norm["Cittadinanza"] != "Mondo"]
# drop total (mashio+femmina) data
resident_foreigners_norm = resident_foreigners_norm[resident_foreigners_norm["Sesso"] != "totale"]

resident_foreigners_norm.index = range(len(resident_foreigners_norm))

In [49]:
resident_norm = resident.copy() 

# drop region and zone data
territori_1 = set(list(regione_zona["Regione"]) + (list(regione_zona["Zona"]) + list(["Italia"]))) 
resident_norm = resident_norm.loc[[i not in territori_1 for i in resident_norm["Territorio"]]]
# drop total (mashio+femmina) data
resident_norm = resident_norm[resident_norm["Sesso"] != "totale"]

resident_norm.index = range(len(resident_norm))

In [50]:
# Provincia is now better than generic Territorio
resident_norm.rename(columns = {"Territorio": "Provincia"}, inplace=True)
resident_foreigners_norm.rename(columns = {"Territorio": "Provincia"}, inplace=True)

In [51]:
resident_norm.head()

,Provincia,Periodo,Sesso,Numero
0,Torino,2013,maschi,1084528
1,Torino,2014,maschi,1109048
2,Torino,2015,maschi,1105608
3,Torino,2016,maschi,1101427
4,Torino,2017,maschi,1100562


In [52]:
resident_foreigners_norm.head()

,Provincia,Cittadinanza,Periodo,Sesso,Numero
0,Alessandria,Andorra,2017,maschi,0
1,Alessandria,Andorra,2017,femmine,1
2,Genova,Andorra,2013,maschi,0
3,Genova,Andorra,2013,femmine,1
4,Como,Andorra,2013,maschi,0


In [53]:
print("We have %d different provinces." %len(list(set(resident_foreigners_norm["Provincia"]))))

We have 109 different provinces.


In [54]:
# dump
resident_norm.to_csv("Data_norm/resident_norm.csv")
resident_foreigners_norm.to_csv("Data_norm/resident_foreigners_norm.csv")